In [1]:
import numpy as np
import pandas as pd
import scipy as sc
import statsmodels as stm

In [2]:
data = pd.read_csv('gene_high_throughput_sequencing.csv', header=0, sep=',')

In [3]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [5]:
genes = data.columns[2:]
genes

Index(['LOC643837', 'LOC100130417', 'SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1',
       'C1orf170', 'HES4', 'ISG15', 'AGRN',
       ...
       'CLIC2', 'RPS4Y1', 'ZFY', 'PRKY', 'USP9Y', 'DDX3Y', 'CD24', 'CYorf15B',
       'KDM5D', 'EIF1AY'],
      dtype='object', length=15748)

In [7]:
from scipy import stats

In [11]:
anses1 = []
anses2 = []
ps1 = []
ps2 = []
for num, gene in enumerate(genes):
    contr = data[data.Diagnosis == 'normal'][gene]
    treat = data[data.Diagnosis == 'early neoplasia'][gene]
    result = stats.ttest_ind(contr, treat, equal_var=False)
    anses1.append(result[1] < 0.05)
    ps1.append(result[1])
    
    contr = treat
    treat = data[data.Diagnosis == 'cancer'][gene]
    result = stats.ttest_ind(contr, treat, equal_var=False)
    anses2.append(result[1] < 0.05)
    ps2.append(result[1])
    if num % 100 == 0:
        print("{} done".format(num))

0 done
100 done
200 done
300 done
400 done
500 done
600 done
700 done
800 done
900 done
1000 done
1100 done
1200 done
1300 done
1400 done
1500 done
1600 done
1700 done
1800 done
1900 done
2000 done
2100 done
2200 done
2300 done
2400 done
2500 done
2600 done
2700 done
2800 done
2900 done
3000 done
3100 done
3200 done
3300 done
3400 done
3500 done
3600 done
3700 done
3800 done
3900 done
4000 done
4100 done
4200 done
4300 done
4400 done
4500 done
4600 done
4700 done
4800 done
4900 done
5000 done
5100 done
5200 done
5300 done
5400 done
5500 done
5600 done
5700 done
5800 done
5900 done
6000 done
6100 done
6200 done
6300 done
6400 done
6500 done
6600 done
6700 done
6800 done
6900 done
7000 done
7100 done
7200 done
7300 done
7400 done
7500 done
7600 done
7700 done
7800 done
7900 done
8000 done
8100 done
8200 done
8300 done
8400 done
8500 done
8600 done
8700 done
8800 done
8900 done
9000 done
9100 done
9200 done
9300 done
9400 done
9500 done
9600 done
9700 done
9800 done
9900 done
10000 done
1

In [10]:
with open('bio1.txt', 'w') as ouf:
    ouf.write(str(sum(anses1)))
    
with open('bio2.txt', 'w') as ouf:
    ouf.write(str(sum(anses2)))
       

In [12]:
def foldChange(contr, treat):
    C = contr.mean()
    T = treat.mean()
    return T / C if T > C else - C / T

In [13]:
genes = np.array(genes)

In [16]:
from statsmodels.stats.multitest import multipletests

In [17]:
reject1, newPs1, _, __ = multipletests(ps1, alpha=0.025, method='holm')

In [22]:
reject2, newPs2, _, __ = multipletests(ps2, alpha=0.025, method='holm')

In [24]:
anses1 = []
anses2 = []
for gene in genes[reject1]:
    contr = data[data.Diagnosis == 'normal'][gene]
    treat = data[data.Diagnosis == 'early neoplasia'][gene]
    if abs(foldChange(contr, treat)) > 1.5:
        anses1.append(gene)
for gene in genes[reject2]:
    contr = data[data.Diagnosis == 'early neoplasia'][gene]
    treat = data[data.Diagnosis == 'cancer'][gene]
    if abs(foldChange(contr, treat)) > 1.5:
        anses2.append(gene)

In [25]:
anses1

['PCSK4', 'EEF1A2']

In [26]:
len(anses2)

77

In [27]:
with open('bio3.txt', 'w') as ouf:
    ouf.write(str(len(anses1)))
    
with open('bio4.txt', 'w') as ouf:
    ouf.write(str(len(anses2)))
       

In [28]:
reject1, newPs1, _, __ = multipletests(ps1, alpha=0.025, method='fdr_bh')
reject2, newPs2, _, __ = multipletests(ps2, alpha=0.025, method='fdr_bh')

In [33]:
anses1 = []
anses2 = []
for gene in genes[reject1]:
    contr = data[data.Diagnosis == 'normal'][gene]
    treat = data[data.Diagnosis == 'early neoplasia'][gene]
    if abs(foldChange(contr, treat)) > 1.5:
        anses1.append(gene)
for gene in genes[reject2]:
    contr = data[data.Diagnosis == 'early neoplasia'][gene]
    treat = data[data.Diagnosis == 'cancer'][gene]
    if abs(foldChange(contr, treat)) > 1.5:
        anses2.append(gene)

In [37]:
with open('bio5.txt', 'w') as ouf:
    ouf.write(str(len(anses1)))
    
with open('bio6.txt', 'w') as ouf:
    ouf.write(str(len(anses2)))
    